In [ ]:
import numpy as np
import pandas as pd
import polars as pl
import os
import gc
import json
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import zarr

gc.enable()

pd.options.display.max_columns = None
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

#pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_str_lengths(10000)

In [ ]:
path = 'I:/Kaggle/czii-cryo-et-object-identification/'

In [ ]:
train_data_experiment_folders_path = path + 'train/static/ExperimentRuns/'
train_data_experiment_folders_path

In [ ]:
test_data_experiment_folders_path = path + 'test/static/ExperimentRuns/'
test_data_experiment_folders_path

In [ ]:
train_data_experiments = os.listdir(train_data_experiment_folders_path)
train_data_experiments

In [ ]:
test_data_experiments = os.listdir(test_data_experiment_folders_path)
test_data_experiments

In [ ]:
data_dict = {}
for experiment in tqdm(train_data_experiments):
    image_types_dict = {}    
    image_types_dict['denoised'] = zarr.open(train_data_experiment_folders_path + f'{experiment}/VoxelSpacing10.000/denoised.zarr', mode='r')
    image_types_dict['iso'] = zarr.open(train_data_experiment_folders_path + f'{experiment}/VoxelSpacing10.000/isonetcorrected.zarr', mode='r')
    image_types_dict['dcon'] = zarr.open(train_data_experiment_folders_path + f'{experiment}/VoxelSpacing10.000/ctfdeconvolved.zarr', mode='r')
    image_types_dict['wbp'] = zarr.open(train_data_experiment_folders_path + f'{experiment}/VoxelSpacing10.000/wbp.zarr', mode='r')
    data_dict[experiment] = image_types_dict

In [ ]:
data_dict

In [ ]:
test_data_dict = {}
for experiment in tqdm(test_data_experiments):
    image_types_dict = {}    
    image_types_dict['denoised'] = zarr.open(test_data_experiment_folders_path + f'{experiment}/VoxelSpacing10.000/denoised.zarr', mode='r')
    test_data_dict[experiment] = image_types_dict

In [ ]:
test_data_dict

In [ ]:
data_dict.keys()

In [ ]:
train_label_experiment_folders_path = path + 'train/overlay/ExperimentRuns/'
train_label_experiment_folders_path

In [ ]:
train_label_experiments = os.listdir(train_label_experiment_folders_path)
train_label_experiments

In [ ]:
labels_dict = {}
for experiment in tqdm(train_label_experiments):
    particle_types_dict = {}
    
    with open(f'{train_label_experiment_folders_path}{experiment}/Picks/apo-ferritin.json') as f:
        loaded_json = json.loads(f.read())
    particle_types_dict['apo-ferritin'] = loaded_json

    '''
    with open(f'{train_label_experiment_folders_path}{experiment}/Picks/beta-amylase.json') as f:
        loaded_json = json.loads(f.read())
    particle_types_dict['beta-amylase'] = loaded_json
    '''

    with open(f'{train_label_experiment_folders_path}{experiment}/Picks/beta-galactosidase.json') as f:
        loaded_json = json.loads(f.read())
    particle_types_dict['beta-galactosidase'] = loaded_json

    with open(f'{train_label_experiment_folders_path}{experiment}/Picks/ribosome.json') as f:
        loaded_json = json.loads(f.read())
    particle_types_dict['ribosome'] = loaded_json

    with open(f'{train_label_experiment_folders_path}{experiment}/Picks/thyroglobulin.json') as f:
        loaded_json = json.loads(f.read())
    particle_types_dict['thyroglobulin'] = loaded_json

    with open(f'{train_label_experiment_folders_path}{experiment}/Picks/virus-like-particle.json') as f:
        loaded_json = json.loads(f.read())
    particle_types_dict['virus-like-particle'] = loaded_json

    labels_dict[experiment] = particle_types_dict

In [ ]:
labels_dict

In [ ]:
experiment_list = []
particle_type_list = []
x_list = []
y_list = []
z_list = []
for experiment in tqdm(test_data_experiments):
    #print(experiment)
    #print(len(labels_dict[experiment]['apo-ferritin']['points']))
    #print(type(labels_dict[experiment]['apo-ferritin']['points']))
    #print(labels_dict[experiment]['apo-ferritin']['points'][0])

    for key in labels_dict[experiment].keys():
        #print(labels_dict[experiment][key])
        #print(labels_dict[experiment][key]['pickable_object_name'])
        for i in range(len(labels_dict[experiment][key]['points'])):
            experiment_list.append(labels_dict[experiment][key]['run_name'])
            particle_type_list.append(labels_dict[experiment][key]['pickable_object_name'])
            x_list.append(labels_dict[experiment][key]['points'][i]['location']['x'])
            y_list.append(labels_dict[experiment][key]['points'][i]['location']['y'])
            z_list.append(labels_dict[experiment][key]['points'][i]['location']['z'])

In [ ]:
print(len(experiment_list))
print(len(particle_type_list))

In [ ]:
labels_df = pd.DataFrame({'experiment':experiment_list, 'particle_type':particle_type_list, 'x':x_list, 'y':y_list, 'z':z_list})
print(labels_df.shape)
labels_df

In [ ]:
labels_df['particle_type'].nunique()

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class YOLO3D(nn.Module):
    def __init__(self, num_classes=5, num_anchors=3):
        super(YOLO3D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(128 * 4 * 4 * 4, num_classes * (7 + num_anchors))  # 3D bounding box output
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

In [ ]:
# Example input: (batch_size=1, channels=1, depth=32, height=32, width=32)
model = YOLO3D()
dummy_input = torch.randn(1, 1, 32, 32, 32)

In [ ]:
output = model(dummy_input)

In [ ]:
output = model(dummy_input)
print(output.shape)  # Should match expected output dimensions


In [ ]:
class YOLO3D(nn.Module):
    def __init__(self, num_classes=5, num_anchors=3):
        super(YOLO3D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)
        self.num_classes = num_classes
        self.num_anchors = num_anchors

        # Placeholder FC layer (will be updated dynamically in forward())
        self.fc = None

    def forward(self, x):
        #print(f"Input shape: {x.shape}")  

        x = torch.relu(self.conv1(x))
        #print(f"After conv1: {x.shape}")

        x = torch.relu(self.conv2(x))
        #print(f"After conv2: {x.shape}")

        x = torch.relu(self.conv3(x))
        #print(f"After conv3: {x.shape}")

        x = torch.flatten(x, start_dim=1)
        #print(f"After flatten: {x.shape}")

        # Dynamically set FC layer the first time forward() runs
        if self.fc is None:
            num_features = x.shape[1]  # Get dynamic input size
            self.fc = nn.Linear(num_features, self.num_classes * (7 + self.num_anchors))
            self.fc = self.fc.to(x.device)  # Move FC layer to the correct device

        x = self.fc(x)
        return x

In [ ]:
# Example input: (batch_size=1, channels=1, depth=32, height=32, width=32)
model = YOLO3D()
dummy_input = torch.randn(1, 1, 32, 32, 32)

In [ ]:
output = model(dummy_input)
print(output.shape)  # Expected shape: (1, num_classes * (7 + num_anchors))

In [ ]:
dummy_input = torch.randn(1, 1, 50, 50, 50)

In [ ]:
output = model(dummy_input)
print(output.shape)  # Expected shape: (1, num_classes * (7 + num_anchors))

In [ ]:
dummy_input.shape

In [ ]:
dummy_input = torch.randn(1, 1, 50, 50, 50)
output = model(dummy_input)
print(output.shape)  # Expected shape: (1, num_classes * (7 + num_anchors))

In [ ]:
dummy_input = torch.randn(1, 1, 60, 60, 60)
output = model(dummy_input)
print(output.shape)  # Expected shape: (1, num_classes * (7 + num_anchors))

In [ ]:
# Example input: (batch_size=1, channels=1, depth=32, height=32, width=32)
model = YOLO3D()
dummy_input = torch.randn(1, 1, 75, 75, 75)
output = model(dummy_input)
print(output.shape)  # Expected shape: (1, num_classes * (7 + num_anchors))

# References

1. https://www.kaggle.com/code/davidlist/experiment-ts-6-4-visualization
2. https://www.kaggle.com/code/nk35jk/3d-visualization-of-particles